In [13]:
# from src.data_examples.ex2_data_loader_1m import ExampleDataLoader
from src.data_examples.ex2_data_loader import ExampleDataLoader

import tqdm
import logging
import pandas as pd
import numpy as np
import random
from pathlib import Path
from src.emulator import UnitProcess, TaskWrapper

pd.options.mode.chained_assignment = None

log_root_path = Path('./log')
log_root_path.mkdir(exist_ok=True)
random.seed(1337)

In [14]:
data_loader = ExampleDataLoader()
data_loader.download().load().remap()

using cached file cache\datasets\movielens\ml-100k.zip
extracting zip file content:
file already exists: data\ex2\ml-100k
file already exists: data\ex2\ml-100k\allbut.pl
file already exists: data\ex2\ml-100k\mku.sh
file already exists: data\ex2\ml-100k\README
file already exists: data\ex2\ml-100k\u.data
file already exists: data\ex2\ml-100k\u.genre
file already exists: data\ex2\ml-100k\u.info
file already exists: data\ex2\ml-100k\u.item
file already exists: data\ex2\ml-100k\u.occupation
file already exists: data\ex2\ml-100k\u.user
file already exists: data\ex2\ml-100k\u1.base
file already exists: data\ex2\ml-100k\u1.test
file already exists: data\ex2\ml-100k\u2.base
file already exists: data\ex2\ml-100k\u2.test
file already exists: data\ex2\ml-100k\u3.base
file already exists: data\ex2\ml-100k\u3.test
file already exists: data\ex2\ml-100k\u4.base
file already exists: data\ex2\ml-100k\u4.test
file already exists: data\ex2\ml-100k\u5.base
file already exists: data\ex2\ml-100k\u5.test
fil

c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column user_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column user_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column movie_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column movie_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column release_date - only the converter will be used.

In [15]:
def init_logger(name, log_path=None):
  formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

  logger = logging.getLogger(name)
  logger.setLevel(logging.DEBUG)

  if not logger.hasHandlers():
    if log_path is None:
      ch = logging.StreamHandler()
    else:
      ch = logging.FileHandler(log_path)
    ch.setFormatter(formatter)
    logger.addHandler(ch)

  return logger


In [16]:
# fixed size units

movie_size = 10e6

num_cl = 20
num_ue = data_loader.df_users['user_id'].nunique()

df_movies = data_loader.df_movies
df_movies['sizeof'] = movie_size
df_movies['movie_id'] = df_movies['movie_id'].astype('int')
df_movies.set_index('movie_id', drop=False, inplace=True)

In [17]:
df_movies

,movie_id,title,release_date,video_release_date,imdb_url,genre_unknown,Action,Adventure,Animation,Children,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,sizeof
movie_id,,,,,,,,,,,,,,,,,,,,,
0,0,Toy Story (1995),1995-01-01,<NA>,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,10000000.0
1,1,GoldenEye (1995),1995-01-01,<NA>,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,10000000.0
2,2,Four Rooms (1995),1995-01-01,<NA>,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,10000000.0
3,3,Get Shorty (1995),1995-01-01,<NA>,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,10000000.0
4,4,Copycat (1995),1995-01-01,<NA>,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,10000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1677,Mat' i syn (1997),1998-02-06,<NA>,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10000000.0
1678,1678,B. Monkey (1998),1998-02-06,<NA>,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,10000000.0
1679,1679,Sliding Doors (1998),1998-01-01,<NA>,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,10000000.0


In [18]:
def handle_clear_cache(tw, host, local_vars):
  tw.log_event('invoked: clear_cache')
  local_vars['disk'].clear()

def handle_cache_overflow(tw, host, local_vars):
  tw.log_event('invoked: cache_overflow')
  host.run_task('handle_disk_usage_calc')
  if local_vars.get('used') > (local_vars.get('alloc') * .9):
    while local_vars.get('used') > (local_vars.get('alloc') * .4):
      host.run_task('handle_clean_cache')

def handle_disk_usage_calc(tw, host, local_vars):
  tw.log_event('invoked: disk_usage_calc')
  disk = local_vars.get('disk')
  local_vars['used'] = sum([i['sizeof'] for i in disk])
  tw.log_event(f'event: recalculated disk usage {host.name}: {local_vars["used"]} / {local_vars["alloc"]}')

def handle_clean_cache_lifo(tw, host, local_vars):
  tw.log_event('invoked: clean_cache_lifo')
  local_vars.get('disk').pop(0)
  host.run_task('handle_disk_usage_calc')

def handle_clean_cache_fifo(tw, host, local_vars):
  tw.log_event('invoked: clean_cache_fifo')
  local_vars.get('disk').pop(-1)
  host.run_task('handle_disk_usage_calc')

def handle_request(tw, host, local_vars, client_name, movie_id):
  tw.log_event(f'invoked: answer_request: {host.name} -> {client_name} | {movie_id}')
  client = host.get_child_by_name(client_name)
  if client is None:
    return

  disk_movie_ids = [i['movie_id'] for i in local_vars['disk']]
  content = None
  if movie_id in disk_movie_ids:
    tw.log_event(f'event: cache hit {client_name} <- {host.name}')
    content = local_vars['disk'].pop(disk_movie_ids.index(movie_id))
  else:
    tw.log_event(f'event: cache missed {client_name} <- {host.name}')
    parent = host
    db = None
    while not parent is None:
      db = parent.local_vars.get('movie_db')
      if db is None:
        parent = parent.parent
      else:
        content = db.loc[movie_id]
        break
  
  local_vars['disk'].append(content)
  host.run_task('handle_cache_overflow')

  client.local_vars['content_buf'] = content

def invoke_content_request(tw, host, local_vars, movie_id, user_id):
  tw.log_event(f'invoked: content_request: {host.name} | {movie_id=} | {user_id=}')
  content = None
  if local_vars.get('local_user_db') is None:
    local_vars['local_user_db'] = dict()
  if local_vars['local_user_db'].get(user_id) is None:
    local_vars['local_user_db'][user_id] = {}
  if local_vars['local_user_db'][user_id].get('request_counter') is None:
    local_vars['local_user_db'][user_id]['request_counter'] = 0
  local_vars['local_user_db'][user_id]['request_counter'] += 1

  disk_movie_ids = [i['movie_id'] for i in local_vars['disk']]
  if movie_id in disk_movie_ids:
    tw.log_event(f'event: cache hit {host.name}')
    content = local_vars['disk'].pop(disk_movie_ids.index(movie_id))
    local_vars['content_buf'] = content
  else:
    tw.log_event(f'event: cache missed {host.name}')
    parent_server = host.parent
    if parent_server is None: return
    tw.log_event(f'event: send_request: {host.name} -> {host.parent.name}')
    parent_server.run_task('handle_request', client_name=host.name, movie_id=movie_id)
    content = local_vars['content_buf']

  local_vars['disk'].append(content)
  host.run_task('handle_cache_overflow')


In [19]:
def handle_logging_shutdown(tw, host, local_vars):
  # tw.log_event(f'invoked: handle_logging_shutdown')
  for handler in tw.host_logger.handlers[:]:
    handler.close()

In [20]:
bs = UnitProcess('BaseServer')
bs.set_logger(init_logger('BaseServer', log_path='log.log'))
bs.local_vars['movie_db'] = df_movies
bs.local_vars.update({
  'alloc': movie_size * 1000,
  'used': 0,
  'disk': list(),
  'content_buf': None,
})

bs.create_task('handle_cache_overflow').set_task(handle_cache_overflow)
bs.create_task('handle_disk_usage_calc').set_task(handle_disk_usage_calc)
bs.create_task('handle_clean_cache').set_task(handle_clean_cache_lifo)
bs.create_task('handle_request').set_task(handle_request)
bs.create_task('handle_logging_shutdown').set_task(handle_logging_shutdown)
bs.create_task('handle_clear_cache').set_task(handle_clear_cache)

In [21]:
for i in range(num_cl):
  name = f'ClientServer [{i:03}]'
  cl = bs.spawn_child(name)
  cl.set_logger(init_logger(name, log_path='log.log'))
  cl.local_vars.update({
    'alloc': movie_size * 10,
    'used': 0,
    'disk': list(),
    'content_buf': None,
  })

  cl.create_task('handle_cache_overflow').set_task(handle_cache_overflow)
  cl.create_task('handle_disk_usage_calc').set_task(handle_disk_usage_calc)
  cl.create_task('handle_clean_cache').set_task(handle_clean_cache_lifo)
  cl.create_task('handle_request').set_task(handle_request)
  cl.create_task('invoke_content_request').set_task(invoke_content_request)
  cl.create_task('handle_logging_shutdown').set_task(handle_logging_shutdown)
  cl.create_task('handle_clear_cache').set_task(handle_clear_cache)


In [22]:
bs.run_task('handle_clear_cache')
for ch in bs.children:
  ch.run_task('handle_clear_cache')

In [23]:
df_req = data_loader.df_ratings.sort_values(by='unix_timestamp')[['user_id', 'movie_id']]

cutoff = 3000
c = 0

for index, row in tqdm.tqdm(df_req.iterrows()):
  movie_id = int(row['movie_id'])
  user_id = row['user_id']
  client = random.choice(bs.children)
  client.run_task('invoke_content_request', movie_id=movie_id, user_id=user_id)

  c += 1
  if c > cutoff: break

0it [00:00, ?it/s]

3000it [00:10, 279.59it/s]


In [24]:
bs.run_task('handle_logging_shutdown')
for ch in bs.children:
  ch.run_task('handle_logging_shutdown')
